Путь к файлу

In [ ]:
file_path = 'rousseau-confessions-119.txt'

In [ ]:
import collections
import re

from os.path import commonprefix
from collections import Counter

import math
import json

from math import ceil

import statistics



Источник утверждает что все файлы в ASCII, проверим это  
Источник файла: https://www.kaggle.com/datasets/mylesoneill/classic-literature-in-ascii

In [ ]:
def is_ascii(file_path):
    try:
        with open(file_path, 'rb') as file:
            content = file.read()
            return all(byte <= 127 for byte in content)
    except IOError:
        print(f"Ошибка: файл не прочитан {file_path}")
        return False


if is_ascii(file_path):
    print("Весь текст в файле находится в формате ASCII")
else:
    print("Файл содержит символы за пределами ASCII")

Весь текст в файле находится в формате ASCII


Выполним очистку файла

In [ ]:
def clean_text(text):
    text = text.replace('\xa0', ' ')   # Неразрывный пробел
    text = text.replace('\u2009', ' ') # Тонкий пробел
    text = text.replace('\u202f', ' ') # Узкий неразрывный пробел
    text = text.encode('ascii', errors='ignore').decode('ascii') # Удаляем все остальные не-ASCII символы (если они есть)
    text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f]', '', text) # Удаляем лишние управляющие символы (оставляем \n)
    text = re.sub(r'\s+', ' ', text).strip() # Заменяем множественные пробелы и табы на один пробел
    return text

In [ ]:
def process_file(input_file, output_file=None):
    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            text = f.read()
        cleaned_text = clean_text(text)
        if output_file:
            with open(output_file, 'w', encoding='ascii') as f:
                f.write(cleaned_text)
            print(f"Текст очищен и сохранён в {output_file}")
        else:
            print("Очищенный текст:\n", cleaned_text[:500] + "...")
    except UnicodeDecodeError:
        print("Ошибка: Файл не в UTF-8")
    except Exception as e:
        print(f"Ошибка: {e}")

In [ ]:
input_file = 'rousseau-confessions-119.txt'
output_file = 'rousseau-confessions-cleaned.txt'
process_file(input_file, output_file)

Текст очищен и сохранён в rousseau-confessions-cleaned.txt


Создаем правила капитализации    
Стандартные правила капитализации: первая буква текста; первая буква после точки с пробелом и/или переводом строки; то же самое для !?; буква, следующая сразу за двумя заглавными; I в окружении не-буквенных символов.

Также исправим **ошибку**: любая буква после пробела должна быть заглавной

In [ ]:
def capitalization(index, text):
    if index < 0 or index >= len(text):
        return False

    ch = text[index]
    if not ch.isalpha():
        return False

    # П1: Первая буква в тексте
    if index == 0:
        return True

    # П2: После .!? И обязательно пробел или перевод строки
    if index > 0:
        j = index - 1
        saw_space_or_newline = False
        while j >= 0 and text[j].isspace(): # .isspace() вкл ' ', '\n', '\t'
            saw_space_or_newline = True
            j -= 1
        # знак конца предложения
        if saw_space_or_newline and j >= 0 and text[j] in '.!?':
            return True

    # П3: Строчная буква после ≥2 заглавных подряд (есть конечно и исключения из этого правила, но они появляются не часто)
    if ch.islower() and index >= 2:
        count = 0
        for i in range(index - 1, -1, -1):
            if text[i].isupper():
                count += 1
            else:
                break
        if count >= 2:
            return True

    # П4: Изолированное 'I'
    if ch == 'I':
        prev = text[index - 1] if index > 0 else ' '
        next = text[index + 1] if index + 1 < len(text) else ' '
        if not prev.isalpha() and not next.isalpha():
            return True
        else:
            return False

    return False

Выполним проверку для перечисленных правил капитализации

In [ ]:
import unittest

class TestCapitalization(unittest.TestCase):

    def test_empty_and_out_of_bounds(self):
        self.assertFalse(capitalization(0, ""), "Пустая строка")
        self.assertFalse(capitalization(-1, "abc"), "Индекс < 0")
        self.assertFalse(capitalization(3, "abc"), "Индекс >= длины")

    def test_non_alpha_chars(self):
        text = " 123.? "
        for i in range(len(text)):
            self.assertFalse(capitalization(i, text), f"Не-буква в позиции {i} ('{text[i]}')")

    def test_rule1_first_letter(self):
        self.assertTrue(capitalization(0, "Apple"), "Первая буква")
        self.assertTrue(capitalization(0, "A"), "Одна буква")

    def test_rule2_after_punctuation(self):
        # T
        self.assertTrue(capitalization(5, "End. Next"), ".<space>N")
        self.assertTrue(capitalization(6, "End!  Next"), "!<space><space>N")
        self.assertTrue(capitalization(5, "End?\nNext"), "?<newline>N")
        self.assertTrue(capitalization(7, "End. \n\tNext"), ".<space><newline><tab>N")
        self.assertTrue(capitalization(2, ". A"), "Начало с .<space>A")

        # F
        self.assertFalse(capitalization(5, "End. 1st Next"), "Не первая буква после точки")
        self.assertFalse(capitalization(0, "."), "Точка не буква")
        self.assertFalse(capitalization(4, "End? No"), "Пробел не буква")
        self.assertTrue(capitalization(0, "A. Next"), "Первая буква A — Правило 1")
        self.assertFalse(capitalization(2, "a. next"), " после точки")

    def test_rule3_after_two_caps(self):
        # T
        self.assertTrue(capitalization(2, "AAa"), "AAa -> a")
        self.assertTrue(capitalization(3, "aBCd"), "aBCd -> d")
        self.assertTrue(capitalization(4, " ABCd"), "<space>ABCd -> d")
        self.assertTrue(capitalization(9, "Start. ABc"), "Start. ABc -> c")

        # F
        self.assertFalse(capitalization(2, "aBc"), "aBc -> c")
        self.assertFalse(capitalization(2, "AaB"), "AaB -> B")
        self.assertTrue(capitalization(0, "AA"), "Индекс 0 (A) - Правило 1")
        self.assertFalse(capitalization(1, "AA"), "Индекс 1 (A) - нет двух перед ним")
        self.assertFalse(capitalization(3, "USA rocks"), "Индекс 4 ('r')")

    def test_rule4_isolated_I(self):
        # T
        self.assertTrue(capitalization(1, " I "), "' I ' -> I")
        self.assertTrue(capitalization(1, "(I)"), "(I) -> I")
        self.assertTrue(capitalization(0, "I."), "I. -> I")
        self.assertTrue(capitalization(0, "I"), "'I' -> I")
        self.assertTrue(capitalization(3, "am I?"), "am I? -> I")
        self.assertTrue(capitalization(1, "\nI\n"), "<nl>I<nl> -> I")

        # F
        self.assertTrue(capitalization(0, "Is"), "'Is' -> I - Правило 1")
        self.assertFalse(capitalization(1, "Is"), "'Is' -> s")
        self.assertFalse(capitalization(2, "Big"), "Big -> g")
        self.assertFalse(capitalization(1, "aI"), "aI -> I")
        self.assertTrue(capitalization(0, "Ib"), "Ib -> I - Правило 1")

    def test_rule_interactions(self):
        self.assertTrue(capitalization(0, "I am."), "Первая 'I'")
        self.assertTrue(capitalization(5, "End. I am"), "После точки 'I'")
        self.assertTrue(capitalization(8, "End. AB I"), "После . и AB -> 'I'")
        self.assertTrue(capitalization(3, "AA I am"), "После AA -> 'I'")

if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestCapitalization)
    runner = unittest.TextTestRunner()
    runner.run(suite)

.......
----------------------------------------------------------------------
Ran 7 tests in 0.003s

OK


In [ ]:
# отдельная проверка на указанную ошибку
text = "I very love tea" # текст с пробелами
index = 2 # буква v в very
capitalization(index, text)

False

Применение правил капитализации к тексту

In [ ]:
# вывод слова в котором из которого мы неправильно определили заглавный символ
def find_word_at_index(text_content, index):
    if not text_content[index].isalpha():
        return ""
    start = index
    while start > 0 and text_content[start - 1].isalpha():
        start -= 1
    end = index
    while end < len(text_content) - 1 and text_content[end + 1].isalpha():
        end += 1
    return text_content[start : end + 1]

In [ ]:
file_name = 'rousseau-confessions-cleaned.txt'
mismatches_count = 0
words_with_mismatches_counts = collections.Counter()

try:
    with open(file_name, 'r', encoding='utf-8') as file:
        text = file.read()
except FileNotFoundError:
    print(f"Ошибка с файлом '{file_name}' не найден")
    exit()
except Exception as e:
    print(f"Ошибка при чтении файла: {e}")
    exit()

if not text:
    print("В файле пусто и грустно")
    exit()

for i, original_char in enumerate(text):
    if original_char.isalpha():
        should_be_capital = capitalization(i, text)
        is_capital_original = original_char.isupper()

        if should_be_capital != is_capital_original:
            mismatches_count += 1
            word = find_word_at_index(text, i)
            if word:
                words_with_mismatches_counts[word] += 1

print(f"Общее количество символов, не совпавших с правилами: {mismatches_count}")

if words_with_mismatches_counts:
    print("\nСлова с ошибкой определения буквы и количество таких ошибок в слове:")
    # Сортируем по словам для вывода
    for word, count in sorted(words_with_mismatches_counts.items()):
        print(f"- {word}: {count}")
else:
    print("Слов с ошибкой нет")


Общее количество символов, не совпавших с правилами: 9020

Слова с ошибкой определения буквы и количество таких ошибок в слове:
- A: 12
- AEsop: 3
- AFTER: 5
- ALTHOUGH: 8
- ANSWER: 12
- Abbe: 77
- Abbes: 1
- Aberdeen: 1
- Abruzzo: 1
- Academy: 8
- According: 1
- Adieu: 1
- Adrastus: 1
- Advent: 1
- African: 1
- Africans: 1
- After: 7
- Agesilaus: 1
- Ah: 14
- Aiguillon: 1
- Aine: 1
- Aix: 3
- Alamanni: 1
- Alary: 1
- Alas: 2
- Albert: 1
- Ale: 1
- Alembert: 25
- Alibard: 1
- All: 2
- Allee: 1
- Alley: 1
- Almighty: 4
- Alpheus: 2
- Alps: 8
- Although: 3
- Altuna: 6
- Amadeus: 2
- Ambassador: 1
- Amelie: 2
- American: 2
- Amours: 1
- Amsterdam: 4
- An: 1
- Anacreon: 1
- Analysis: 1
- Anaxarete: 1
- Ancelet: 1
- And: 4
- Andilly: 2
- Andiol: 8
- Anet: 25
- Angers: 1
- Annecy: 29
- Antoines: 1
- Antonine: 2
- Antremont: 5
- Anzoletta: 1
- Apollo: 3
- Apparently: 1
- Arberg: 1
- Arbois: 1
- Arbourg: 1
- Archimandrite: 7
- Arethusa: 2
- Argenson: 3
- Arguses: 1
- Aristides: 2
- Armenian: 4

In [ ]:
def sort_dict_by_values(input_dict, reverse=True):
    if not isinstance(input_dict, dict):
        raise TypeError("Входной аргумент должен быть словарем.")
    sorted_items = sorted(input_dict.items(), key=lambda item: item[1], reverse=reverse)
    return sorted_items
sort_dict_by_values(words_with_mismatches_counts)

[('Madam', 645),
 ('M', 500),
 ('de', 291),
 ('Paris', 200),
 ('Warrens', 176),
 ('Luxembourg', 134),
 ('Epinay', 130),
 ('Geneva', 118),
 ('Theresa', 88),
 ('France', 80),
 ('Diderot', 80),
 ('Grimm', 79),
 ('Abbe', 77),
 ('Houdetot', 77),
 ('Saint', 74),
 ('MONTMORENCY', 73),
 ('Hermitage', 70),
 ('French', 65),
 ('Montmorency', 64),
 ('St', 62),
 ('Mademoiselle', 57),
 ('D', 57),
 ('Dupin', 45),
 ('The', 44),
 ('Venice', 43),
 ('Chambery', 42),
 ('Lambert', 39),
 ('BOOK', 38),
 ('Lyons', 38),
 ('Count', 37),
 ('Larnage', 36),
 ('Maitre', 35),
 ('Father', 34),
 ('le', 34),
 ('Comte', 33),
 ('Francueil', 33),
 ('Motiers', 32),
 ('d', 31),
 ('La', 30),
 ('Montaigu', 30),
 ('Miss', 29),
 ('Annecy', 29),
 ('Vasseur', 29),
 ('Boufflers', 29),
 ('Italian', 28),
 ('This', 28),
 ('Chevrette', 28),
 ('King', 27),
 ('Le', 26),
 ('Monsieur', 26),
 ('Anet', 25),
 ('De', 25),
 ('Alembert', 25),
 ('Neuchatel', 25),
 ('Heaven', 24),
 ('Malesherbes', 24),
 ('Emile', 24),
 ('Turin', 23),
 ('Marquis',

Посмотрев на список слов, можно заметить в нем наличие имен собственных в разных склонениях, однако их не так много. А также опечатки/либо авторский замысел: AFTER, AEsop и д.р. В тексте всего 1516271 символов.

С помощью spaCy и конрпуса французского языка можно посчитать количество более сложных именованных сущностей. Буду использовать модель fr_core_news_md и fr_core_news_lg. Здесь будут только результаты, чтобы не было лишнего.

Были найдены следующие имена и их частота:   
'Madam de Warrens': 73    
'Diderot': 41   
'Madam de Luxembourg': 24   
'Madam Dupin': 21   
'Paris': 20  
'Grimm': 17  
'Geneva': 16  
'Theresa': 16   
'Madam d'Epinay': 16   
'Voltaire': 11  

Остальные имена собственные повторяются реже.

Объединив результаты, можно получить следующий список имен (если имя появляется чаще 45 раз, за исключением слова Madam в обращениях)

'Paris',
'Warrens',
'Luxembourg',
'Epinay',
'Geneva',
'Theresa',
'France',
'Diderot',
'Grimm',
'Abbe',
'Houdetot',
'Saint',
'MONTMORENCY',
'Hermitage',
'French',
'Montmorency',
'St',
'Mademoiselle',
'Dupin',
'Madam de Warrens',
'Diderot',
'Madam d'Epinay',
'Madam de Luxembourg',
'Madam Dupin'

In [ ]:
SPECIAL_CASES = [
    'Paris', 'Warrens', 'Luxembourg', 'Epinay', 'Geneva', 'Theresa',
    'France', 'Diderot', 'Grimm', 'Abbe', 'Houdetot', 'Hermitage', 'French',
    'Mademoiselle', 'Dupin', 'Madam de Warrens','Madam de Dupin'
]
# добавим правило
def capitalization(index, text, special_cases_list):
    if index < 0 or index >= len(text):
        return False

    ch = text[index]
    if not ch.isalpha():
        return False

    # П1: Первая буква в тексте
    if index == 0:
        return True
    # П* правило для специальных слов/фраз
    for special_phrase in special_cases_list:
        for phrase_char_idx in range(len(special_phrase)):
            text_start_idx_for_match = index - phrase_char_idx
            if text_start_idx_for_match >= 0 and \
               text_start_idx_for_match + len(special_phrase) <= len(text):
                segment_in_text = text[text_start_idx_for_match : text_start_idx_for_match + len(special_phrase)]
                if segment_in_text.lower() == special_phrase.lower():
                    return special_phrase[phrase_char_idx].isupper()
    # П2: После .!? И обязательно пробел или перевод строки
    if index > 0:
        j = index - 1
        saw_space_or_newline = False
        while j >= 0 and text[j].isspace(): # .isspace() вкл ' ', '\n', '\t'
            saw_space_or_newline = True
            j -= 1
        # знак конца предложения
        if saw_space_or_newline and j >= 0 and text[j] in '.!?':
            return True

    # П3: Строчная буква после ≥2 заглавных подряд (есть конечно и исключения из этого правила, но они появляются не часто)
    if ch.islower() and index >= 2:
        count = 0
        for i in range(index - 1, -1, -1):
            if text[i].isupper():
                count += 1
            else:
                break
        if count >= 2:
            return True

    # П4: Изолированное 'I'
    if ch == 'I':
        prev = text[index - 1] if index > 0 else ' '
        next = text[index + 1] if index + 1 < len(text) else ' '
        if not prev.isalpha() and not next.isalpha():
            return True
        else:
            return False

    return False

In [ ]:
file_name = 'rousseau-confessions-cleaned.txt'
mismatches_count = 0
words_with_mismatches_counts = collections.Counter()

try:
    with open(file_name, 'r', encoding='utf-8') as file:
        text = file.read()
except FileNotFoundError:
    print(f"Ошибка с файлом '{file_name}' не найден")
    exit()
except Exception as e:
    print(f"Ошибка при чтении файла: {e}")
    exit()

if not text:
    print("В файле пусто и грустно")
    exit()

for i, original_char in enumerate(text):
    if original_char.isalpha():
        should_be_capital = capitalization(i, text, SPECIAL_CASES)
        is_capital_original = original_char.isupper()

        if should_be_capital != is_capital_original:
            mismatches_count += 1
            word = find_word_at_index(text, i)
            if word:
                words_with_mismatches_counts[word] += 1

print(f"Общее количество символов, не совпавших с правилами: {mismatches_count}")

if words_with_mismatches_counts:
    print("\nСлова с ошибкой определения буквы и количество таких ошибок в слове:")
    # Сортируем по словам для вывода
    for word, count in sorted(words_with_mismatches_counts.items()):
        print(f"- {word}: {count}")
else:
    print("Слов с ошибкой нет")


Общее количество символов, не совпавших с правилами: 7401

Слова с ошибкой определения буквы и количество таких ошибок в слове:
- A: 12
- AEsop: 3
- AFTER: 5
- ALTHOUGH: 8
- ANSWER: 12
- Aberdeen: 1
- Abruzzo: 1
- Academy: 8
- According: 1
- Adieu: 1
- Adrastus: 1
- Advent: 1
- African: 1
- Africans: 1
- After: 7
- Agesilaus: 1
- Ah: 14
- Aiguillon: 1
- Aine: 1
- Aix: 3
- Alamanni: 1
- Alary: 1
- Alas: 2
- Albert: 1
- Ale: 1
- Alembert: 25
- Alibard: 1
- All: 2
- Allee: 1
- Alley: 1
- Almighty: 4
- Alpheus: 2
- Alps: 8
- Although: 3
- Altuna: 6
- Amadeus: 2
- Ambassador: 1
- Amelie: 2
- American: 2
- Amours: 1
- Amsterdam: 4
- An: 1
- Anacreon: 1
- Analysis: 1
- Anaxarete: 1
- Ancelet: 1
- And: 4
- Andilly: 2
- Andiol: 8
- Anet: 25
- Angers: 1
- Annecy: 29
- Antoines: 1
- Antonine: 2
- Antremont: 5
- Anzoletta: 1
- Apollo: 3
- Apparently: 1
- Arberg: 1
- Arbois: 1
- Arbourg: 1
- Archimandrite: 7
- Arethusa: 2
- Argenson: 3
- Arguses: 1
- Aristides: 2
- Armenian: 4
- Armentieres: 1
- Ar

Как можно заметить, количество исключений сократьлось на 2000 примерно.

Можно запоминать расстояние между исключениями, которые не описываются правилами и впоследствии закодировать их. Для начала получим битовый массив, в котором 1 будут на позиции символа, написание которого отклоняетася от правил. Скорее всего массив будет разряжен, поэтому для его сокращения будем запоминать расстояния между 1 (единицами = исключениями)./

In [ ]:
def exeption_counter(text, SPECIAL_CASES):
    mismatches_count = 0
    exception_markers = [0] * len(text)
    for i, original_char in enumerate(text):
        if original_char.isalpha():
            should_be_capital = capitalization(i, text, SPECIAL_CASES)
            is_capital_original = original_char.isupper()

            if should_be_capital != is_capital_original:
                exception_markers[i] = 1 # исключение
                mismatches_count += 1
                word = find_word_at_index(text, i)
                if word:
                    words_with_mismatches_counts[word] += 1
    print(f"Общее количество символов, не совпавших с правилами: {mismatches_count}")
    print(f"Длина массива исключений: {len(exception_markers)}")
    if len(exception_markers) > 100:
        print(f"Первые 100 элементов: {exception_markers[:100]}")
        print(f"Последние 100 элементов: {exception_markers[-100:]}")
    else:
        print(exception_markers)
    return exception_markers

Простой тест

In [ ]:
text = "I Love. math" # позиции исключений: 2, 8
exeption_counter(text, SPECIAL_CASES)

Общее количество символов, не совпавших с правилами: 2
Длина массива исключений: 12
[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]


[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]

Построим битовый массив для нашего текста

In [ ]:
file_name = 'rousseau-confessions-cleaned.txt'
mismatches_count = 0
words_with_mismatches_counts = collections.Counter()

try:
    with open(file_name, 'r', encoding='utf-8') as file:
        text = file.read()
except FileNotFoundError:
    print(f"Ошибка с файлом '{file_name}' не найден")
    exit()
except Exception as e:
    print(f"Ошибка при чтении файла: {e}")
    exit()

if not text:
    print("В файле пусто и грустно")
    exit()

exception_list = exeption_counter(text, SPECIAL_CASES)
#exception_list

Общее количество символов, не совпавших с правилами: 7401
Длина массива исключений: 1516271
Первые 100 элементов: [0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
Последние 100 элементов: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0]


Функция для создания массива расстояний между исключениями

In [ ]:
def exeptions_dist(exception_markers):
    distances = []
    last_one_index = -1
    for i, marker in enumerate(exception_markers):
        if marker == 1:
            if last_one_index == -1: # Первая единица
                distances.append(i)
            else:
                distances.append(i - last_one_index)
            last_one_index = i

    print("\nМассив расстояний между исключениями")
    if not distances:
        print("Исключения не найдены, массив расстояний пуст.")
    else:
        # print(distances)
        print(f"Количество исключений (длина массива расстояний): {len(distances)}")
        if len(distances) > 100:
            print(f"Первые 100 расстояний: {distances[:100]}")
            print(f"Последние 100 расстояний: {distances[-100:]}")
        else:
            print(distances)
    return distances

Простая проверка

In [ ]:
arr = [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]
exeptions_dist(arr)


Массив расстояний между исключениями
Количество исключений (длина массива расстояний): 2
[2, 6]


[2, 6]

Получим массив расстояний для нашего текста

In [ ]:
distance_list = exeptions_dist(exception_list)


Массив расстояний между исключениями
Количество исключений (длина массива расстояний): 7401
Первые 100 расстояний: [5, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 5, 5, 8, 23, 13, 8, 1, 1, 1, 18, 1, 1, 1, 385, 198, 10, 55, 544, 31, 6, 9, 15, 338, 6, 13, 9, 299, 314, 861, 49, 63, 8, 483, 62, 15, 7, 66, 89, 52, 124, 6, 6, 9, 426, 436, 37, 38, 29, 33, 64, 34, 27, 35, 35, 684, 5, 59, 98, 36, 156, 233, 913, 1063, 1130, 11]
Последние 100 расстояний: [34, 621, 631, 1510, 149, 275, 572, 26, 255, 189, 585, 172, 158, 90, 33, 43, 90, 555, 353, 60, 80, 3, 3, 88, 283, 308, 43, 53, 355, 63, 14, 280, 32, 42, 62, 184, 64, 160, 149, 207, 64, 3, 3, 71, 82, 125, 19, 84, 8, 55, 281, 403, 10, 30, 199, 187, 3, 3, 182, 181, 203, 31, 16, 20, 665, 660, 815, 146, 3, 3, 4, 11, 366, 185, 301, 238, 613, 225, 39, 130, 159, 3, 4, 11, 12, 12, 7, 16, 15, 15, 11, 601, 47, 8, 196, 1, 1, 2, 1, 1]


Проанализируем полученный массив

In [ ]:

def analyze_distance_list(distance_list):
    if not distance_list:
        return
    num_distances = len(distance_list)
    print(f"1. Общее количество расстояний (исключений): {num_distances}")

    min_dist = min(distance_list)
    max_dist = max(distance_list)
    print(f"2. Минимальное расстояние: {min_dist}")
    print(f"3. Максимальное расстояние: {max_dist}")

    range_dist = max_dist - min_dist
    print(f"4. Размах значений (Max - Min): {range_dist}")

    mean_dist = statistics.mean(distance_list)
    print(f"6. Среднее арифметическое расстояние: {mean_dist:.2f}")

    median_dist = statistics.median(distance_list)
    print(f"7. Медианное расстояние: {median_dist}")

    if num_distances > 1:
        stdev_dist = statistics.stdev(distance_list)
        print(f"8. Стандартное отклонение: {stdev_dist:.2f}")
    else:
        print("8. Стандартное отклонение: недостаточно данных (нужно > 1 значения).")
    try:
        modes = statistics.multimode(distance_list)
        if len(modes) == num_distances and num_distances > 1: # Все значения уникальны
             print("9. Мода: все значения уникальны.")
        else:
            print(f"9. Мода (наиболее частые значения): {modes}")
    except statistics.StatisticsError:
        print("9. Мода: не удалось вычислить (возможно, все значения уникальны).")

    try:
        quartiles = statistics.quantiles(distance_list, n=4, method='exclusive') # Q1, Q2 (медиана), Q3
        print(f"   - 25-й перцентиль (Q1): {quartiles[0]}")
        print(f"   - 50-й перцентиль (Q2, Медиана): {quartiles[1]}")
        print(f"   - 75-й перцентиль (Q3): {quartiles[2]}")

    except statistics.StatisticsError:
        print("   - Недостаточно данных для расчета квантилей.")
    except IndexError:
        print("   - Недостаточно данных для расчета всех запрашиваемых квантилей.")


    print("\n11. Частотное распределение (топ 10 самых частых расстояний):")
    frequencies = collections.Counter(distance_list)
    most_common_distances = frequencies.most_common(10)
    for value, count in most_common_distances:
        print(f"   - Расстояние {value}: встречается {count} раз ({(count/num_distances)*100:.2f}%)")

    if len(frequencies) > 10:
        print(f"   ... и еще {len(frequencies) - 10} уникальных значений расстояний.")
    print(f"   Всего уникальных значений расстояний: {len(frequencies)}")


In [ ]:
analyze_distance_list(distance_list)

1. Общее количество расстояний (исключений): 7401
2. Минимальное расстояние: 1
3. Максимальное расстояние: 5923
4. Размах значений (Max - Min): 5922
6. Среднее арифметическое расстояние: 204.87
7. Медианное расстояние: 49
8. Стандартное отклонение: 393.88
9. Мода (наиболее частые значения): [3]
   - 25-й перцентиль (Q1): 9.0
   - 50-й перцентиль (Q2, Медиана): 49.0
   - 75-й перцентиль (Q3): 228.0

11. Частотное распределение (топ 10 самых частых расстояний):
   - Расстояние 3: встречается 706 раз (9.54%)
   - Расстояние 9: встречается 358 раз (4.84%)
   - Расстояние 1: встречается 306 раз (4.13%)
   - Расстояние 6: встречается 216 раз (2.92%)
   - Расстояние 8: встречается 166 раз (2.24%)
   - Расстояние 7: встречается 136 раз (1.84%)
   - Расстояние 10: встречается 128 раз (1.73%)
   - Расстояние 5: встречается 117 раз (1.58%)
   - Расстояние 11: встречается 107 раз (1.45%)
   - Расстояние 12: встречается 105 раз (1.42%)
   ... и еще 1081 уникальных значений расстояний.
   Всего уник

Выводы: достаточно большой диапазон и разброс занчений, большая часть расстояний не превышает 228 (75%), при этом есть выбросы - какие-то очень большие значения. Существует довольно много уникальных значений - 1091 из 7401. Стоит рассмотреть  статистические методы. Т.е. либо Хаффман, либо арифметическое кодирование. Учитывая, что большинство значений не велики коды Элиаса (gamma, delta) могут быть эффективны. Также стоит заметить, что при визуальном анализе в массиве не так много повторов и "контекстно зависимых" элементов, однако PPM  стоит сравнить с другими методами.

Размер массива с единицами на местах исключений:  
* Общее количество символов, не совпавших с правилами: 7401
* Длина массива исключений: 1516271
* 1,516,271 бит/8 бит/байт=189,533.875 байт
* 189,533.875 байт/1024 байт/КБ≈185.09 КБ   
Мы должны используя сжатие получить меньше 185.09 КБ

Статическое арифметическое кодирование. Проверим на примере из лекции

In [ ]:
from math import ceil, floor
from os.path import commonprefix
from collections import Counter

def bin_line(n : int, N: int) -> str:
    return "0" * (N - len(bin(n)[2:])) + bin(n)[2:]


def get_l_1 (l, h, a, N):
    return l + ceil((h - l + 1) * a / (2 ** N))

def get_h_1 (l, h, b, N):
    return l + ceil((h - l + 1) * (b + 1) / (2 ** N)) - 1

def count_frequencies(source_message : list) -> dict:
    return dict(Counter(source_message))


def create_intervals(frequencies : dict, N : int, source_message_len : int) -> dict:
    intervals = dict()
    current_left_value = 0
    current_prob = 0
    for letter in frequencies.keys():
        current_prob += frequencies[letter]
        intervals[letter] = [current_left_value,
                             ceil(current_prob * (2 ** N) / source_message_len) - 1]
        current_left_value = ceil(current_prob * (2 ** N) / source_message_len)
    return intervals

def get_matched_bits(l : int, h : int, N) -> str:
    l_bin = bin_line(l, N)
    h_bin = bin_line(h, N)

    return commonprefix([l_bin, h_bin])


def count_l_new(l_1, matched_bits, N):
    to_add = len(matched_bits) * "0"
    l_bin = bin_line(l_1, N)
    return int(l_bin[len(matched_bits):] + to_add, 2)

def count_h_new(h_1, matched_bits, N):
    to_add = len(matched_bits) * "1"
    h_bin = bin_line(h_1, N)
    return int(h_bin[len(matched_bits):] + to_add, 2)

def count_r(x : str) -> int:
    inverse = "0"
    if x[0] == "0":
        inverse = "1"
    index = 1
    while (index < len(x) and x[index] == inverse):
        index+=1
    return index - 1

def trick(l : int, h : int, N) -> tuple:
    l_bin = bin_line(l, N)
    h_bin = bin_line(h, N)
    # if l_bin[0] == "0" and l_bin[1] == "1" and h_bin[0] == "1" and h_bin[1] == "0":
    if get_matched_bits(l, h, N):
        return l, h

    r = min(count_r(l_bin), count_r(h_bin))
    l_bin = "0" + l_bin[r+1:] + "0" * r
    h_bin = "1" + h_bin[r+1:] + "1" * r
    # print("computed r =", r)
    return int(l_bin, 2), int(h_bin, 2), r


def bits_to_add_with_bits(matched_bits : str, bits : int) -> str:
    inverse = "0"
    if matched_bits[0] == "0":
        inverse = "1"
    return matched_bits[0] + inverse * bits + matched_bits[1:]

def compress(source_message : list, N : int, frequencies : dict, intervals, dynamic : bool=False):
    l = 0
    h = 2 ** N - 1
    encoded_sequence = ''
    bits = 0
    not_changed_counter = 0
    for index, letter in enumerate(source_message):
        l_1 = get_l_1(l, h, intervals[letter][0], N)
        h_1 = get_h_1(l, h, intervals[letter][1], N)
        matched_bits = get_matched_bits(l_1, h_1, N)
        l_new = count_l_new(l_1, matched_bits, N)
        h_new = count_h_new(h_1, matched_bits, N)
        if matched_bits:
            encoded_sequence += bits_to_add_with_bits(matched_bits, bits)
            bits = 0
        l_new, h_new, new_bits = trick(l_new, h_new, N)
        if new_bits != 0:
            bits += new_bits
        if matched_bits == "" and bits == 0:
            not_changed_counter += 1
        # print_state(index, letter, l, h, l_1, h_1, l_new, h_new, enc, matched_bits, bits, not_changed_counter, intervals)
        l, h = l_new, h_new
        if dynamic:
            frequencies[letter] += 1
            intervals = create_intervals(frequencies, N, len(frequencies.keys()) + index + 1)
    return encoded_sequence + "1" + not_changed_counter * "0"

def static_arithmetic_compression(source_message:list, bit_precision_config:int):
    frequencies = count_frequencies(source_message)
    intervals = create_intervals(frequencies, bit_precision_config, len(source_message))
    return compress(source_message, bit_precision_config, frequencies, intervals, dynamic=False)

Динамическое арифметическое кодирование

In [ ]:
def din_arithmetic_compression(source_message:list, bit_precision_config:int):
    frequencies = {i : 1 for i in source_message}
    intervals = create_intervals(frequencies, bit_precision_config, len(frequencies.keys()))
    return compress(source_message, bit_precision_config, frequencies, intervals, dynamic=True)

Простой тест по примеру из лекции

In [ ]:
#source_message = "acagaatagaga"
source_message = ["A", "C", "A", "G", "A", "A","T", "A", "G", "A", "G", "A"]
bit_precision_config = 8

print(f"Original Text: {source_message}")
compressed_output = static_arithmetic_compression(list(source_message), bit_precision_config)
print(f"Encoded Output: {compressed_output}")

Original Text: ['A', 'C', 'A', 'G', 'A', 'A', 'T', 'A', 'G', 'A', 'G', 'A']
Encoded Output: 0101110110111001000


Пример из домашки. Только список символолв не задается явно

In [ ]:
source_message = ["A", "C", "C", "A", "C", "C","A", "T", "G", "G", "T"]
bit_precision_config = 8

print(f"Original Text: {source_message}")
compressed_output = din_arithmetic_compression(list(source_message), bit_precision_config)
print(f"Encoded Output: {compressed_output}")

Original Text: ['A', 'C', 'C', 'A', 'C', 'C', 'A', 'T', 'G', 'G', 'T']
Encoded Output: 0001111010111111100111110


Закодируем массив расстояний

In [ ]:
bit_precision_config = 32
source_message = distance_list
print(f"Массив расстояний: {source_message}")
compressed_output = static_arithmetic_compression(list(source_message), bit_precision_config)
print(f"Закодированный массив: {compressed_output}")

Массив расстояний: [5, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 5, 5, 8, 23, 13, 8, 1, 1, 1, 18, 1, 1, 1, 385, 198, 10, 55, 544, 31, 6, 9, 15, 338, 6, 13, 9, 299, 314, 861, 49, 63, 8, 483, 62, 15, 7, 66, 89, 52, 124, 6, 6, 9, 426, 436, 37, 38, 29, 33, 64, 34, 27, 35, 35, 684, 5, 59, 98, 36, 156, 233, 913, 1063, 1130, 11, 10, 3, 7, 11, 14, 10, 9, 11, 11, 11, 10, 6, 7, 15, 3, 9, 13, 11, 17, 27, 378, 11, 12, 14, 11, 15, 397, 9, 227, 9, 298, 161, 1196, 982, 6, 119, 6, 884, 1719, 17, 12, 10, 8, 8, 16, 40, 30, 581, 1038, 166, 26, 9, 34, 142, 170, 1194, 1339, 399, 1001, 5, 337, 5, 605, 5, 952, 5, 346, 5, 1082, 5, 360, 379, 6, 1034, 5, 1288, 1492, 5, 285, 4, 5, 409, 5, 21, 992, 1667, 1543, 76, 10, 1037, 4, 4, 5, 103, 780, 748, 5, 109, 8, 798, 2094, 1035, 422, 298, 58, 358, 335, 85, 10, 271, 274, 721, 1752, 37, 7, 329, 61, 1145, 6, 94, 6, 11, 359, 6, 365, 26, 208, 9, 347, 5, 612, 5, 118, 5, 186, 5, 204, 1037, 5, 640, 5, 514, 5, 78, 11

Длина битового закодированного массива

In [ ]:
len(compressed_output)

59540

Для статического кодирования нужен список частот (его тоже можно закодировать)

In [ ]:
import pickle

def measure_frequency_table_size_kb(data: list) -> float:
    frequency_table = count_frequencies(data)
    serialized = pickle.dumps(frequency_table)
    # print(serialized)
    return len(serialized) / 1024, len(static_arithmetic_compression(list(serialized), 32))/1024/8
size_kb, compress_size_kb = measure_frequency_table_size_kb(distance_list)
print(f"Размер таблицы частот: {size_kb:.3f} КБ")
print(f"Размер таблицы частот сжатой: {compress_size_kb:.3f} КБ")

Размер таблицы частот: 5.100 КБ
Размер таблицы частот сжатой: 2.963 КБ


Получается, что длинна битового закодированного массива (статическое арифметическое кодирование) (расстояние между исключениями)= 7 kB + 2.928 kB = 9.9 kB

Посмотрим на размер при использовании динамического кодирования

In [ ]:
bit_precision_config = 32
source_message = distance_list
print(f"Массив расстояний: {source_message}")
compressed_output = din_arithmetic_compression(source_message, bit_precision_config)
print(f"Закодированный массив: {compressed_output}")

Массив расстояний: [5, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 5, 5, 8, 23, 13, 8, 1, 1, 1, 18, 1, 1, 1, 385, 198, 10, 55, 544, 31, 6, 9, 15, 338, 6, 13, 9, 299, 314, 861, 49, 63, 8, 483, 62, 15, 7, 66, 89, 52, 124, 6, 6, 9, 426, 436, 37, 38, 29, 33, 64, 34, 27, 35, 35, 684, 5, 59, 98, 36, 156, 233, 913, 1063, 1130, 11, 10, 3, 7, 11, 14, 10, 9, 11, 11, 11, 10, 6, 7, 15, 3, 9, 13, 11, 17, 27, 378, 11, 12, 14, 11, 15, 397, 9, 227, 9, 298, 161, 1196, 982, 6, 119, 6, 884, 1719, 17, 12, 10, 8, 8, 16, 40, 30, 581, 1038, 166, 26, 9, 34, 142, 170, 1194, 1339, 399, 1001, 5, 337, 5, 605, 5, 952, 5, 346, 5, 1082, 5, 360, 379, 6, 1034, 5, 1288, 1492, 5, 285, 4, 5, 409, 5, 21, 992, 1667, 1543, 76, 10, 1037, 4, 4, 5, 103, 780, 748, 5, 109, 8, 798, 2094, 1035, 422, 298, 58, 358, 335, 85, 10, 271, 274, 721, 1752, 37, 7, 329, 61, 1145, 6, 94, 6, 11, 359, 6, 365, 26, 208, 9, 347, 5, 612, 5, 118, 5, 186, 5, 204, 1037, 5, 640, 5, 514, 5, 78, 11

In [ ]:
len(compressed_output)

61960

In [ ]:
len(compressed_output)/8/1024

7.5634765625

Получается, что длинна битового закодированного массива (динамическое арифметическое кодирование) (расстояние между исключениями)= 7.56 kB

Можно также оценить затраты на передачу имен собственных

In [ ]:
delimiter = b'\x00'
names_bytes  = bytearray()
for s in SPECIAL_CASES:
    names_bytes.extend(s.encode('utf-8') + delimiter)
len(names_bytes)

150

Т.е. для передачи имен собственных потребуется 150 байт, при этом добавление правила основанного на них сокращает список исключений на 2000 пунктов

Рассмотрим алгоритм сжатия PPM. Возьмем готовую реализацию https://github.com/miurahr/pyppmd. pyppmd.compress

In [ ]:
%pip install pyppmd

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pyppmd
import struct

data_bytes = b''.join(struct.pack('H', num) for num in distance_list)
compressed_output = pyppmd.compress(data_bytes, max_order=6, mem_size=8, variant="I")
print(f"Закодированный PPM массив: {compressed_output}")

Закодированный PPM массив: b'\x04\xfb\t\xb4\xbd\xbf\xb9M\xa4\xee\xdb\xa9\x13\x10\xa9\xc40\xce\xf2\xc9\xcd\xd2Id4-\xf3\x060|G\xa0e\x89\x12\xc8\xf0\x8a\xae\x9c\xb9\xc4R\xd35\xc6j\xc6\x9a:\x91\xafq\xb4\xf9Z\x12s\x16\xc0\t\x0cD>T\xf9\x05\x06I\xa5r+\x00\x81N\xac=\xf9\xd3\xb8\x9d\r\xd9<\x14\xc6\x81\xdd\xebP\xddv6\xbf\xad\xe1\xd3\xba\xe0\xeb\x1b0e\xe3\x01\x0c\x17\x97\x06{\x81\x8f\x08\xae\x1dC\xab<9\xa5Q\x8e\x9f^S\xd0I\xbe\x9cx\xb4\xb0\x19TN\x11%\x8bj5\xcc|\x9c\xd2\xb7\xb0\xadI\x0e\x1d\xeeO;\xce\x06~\xf0!\xe4>\xc5\\\xac\x1a\x1a\xad\x11\xd1\xce\x05m^p\t\xe9\xd8\xa0\x12W\xb8\x8c\xaa\xd7\n\xe3a~\xef\xbf[iL\xec$R07\xb5\xaf\xc8_\xd0\x99D8\xf1\x95\n\x9b\xa9\xf1\xd5By\xc3\xf0x\xab\xe3\x89\t\xf4\xf6\xc2\xf5\x03\xd2\xaf\xc7\x08\x87\x06g-T\x8b\x05\xfa{\xeb|\xf3.\x0ec\xc7\x98O\x88\x80\xde\xb4\xd2\xaa\xf09\x87D\xd1#\x0f\xa6K\xec\xd39\xd5Fr\xe7\xa74\xcf\xe7z\xb3MM\xee\xd7\x83\xb14y\xc4T\xc9\x1c\xfe\xde\xf8\xc9\xb0t\x92k\xf5^\xd8+\xbb\x06\xfaaN\xd3\xaa\x1c}U\x12\x8b\xdb\x00\x82\x94\x00\xa2;\xd9\xc9\x8e\xb6\

In [ ]:
len(compressed_output) / 1024

11.0625

Размер сжатого массива расстояний (метод PPM) = 11.0625 KB

Также используя готовые решения посмотрим на размер массива, сжатого с помощью Хаффмана и кодов Элиаса

In [ ]:
%pip install huffman

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import huffman

# Создаем частотный словарь
freq_dict = {}
for num in distance_list:
    freq_dict[num] = freq_dict.get(num, 0) + 1
codebook = huffman.codebook(freq_dict.items())

# кодируем массив расстояний массив
encoded_bits = ''.join([codebook[num] for num in distance_list])

# преобразование в байты
encoded_bytes = int(encoded_bits, 2).to_bytes((len(encoded_bits) + 7) // 8, 'big')

print("Закодированные данные (биты):", encoded_bits)
print(f"Размер сжатых данных: {len(encoded_bits)/1024/8:.2f} КБ")

Закодированные данные (биты): ...
Размер сжатых данных: 7.30 КБ


Дельта и гамма-кодирование Элиаса

Простой пример для проверки

In [ ]:
# distance_list = [1,4,2] # гамма 1 00100 010
#                         # дельта 1 01100 010

In [ ]:
from math import log, floor

def binary_without_msb(x):
    binary = bin(x)[2:]
    return binary[1:] if len(binary) > 1 else ''

def elias_gamma_encode(k):
    if k == 0:
        return '0'
    n = 1 + floor(log(k, 2))
    unary = (n-1)*'0' + '1'
    return unary + binary_without_msb(k)

def elias_delta_encode(k):
    if k == 0:
        return '0'
    length = 1 + floor(log(k, 2))
    gamma_part = elias_gamma_encode(length)
    return gamma_part + binary_without_msb(k)

def encode_array(numbers, encoding_func):
    encoded_bits = ''.join(encoding_func(num) for num in numbers)

    # длина была кратна 8 битам
    padding_length = (8 - len(encoded_bits) % 8) % 8
    encoded_bits += '0' * padding_length

    byte_array = bytes(
        int(encoded_bits[i:i+8], 2)
        for i in range(0, len(encoded_bits), 8)
    )

    return byte_array, padding_length


gamma_encoded, gamma_padding = encode_array(distance_list, elias_gamma_encode)
delta_encoded, delta_padding = encode_array(distance_list, elias_delta_encode)

# размер в килобайтах
gamma_size_kb = len(gamma_encoded) / 1024
delta_size_kb = len(delta_encoded) / 1024

print(f"Гамма-кодирование: размер {gamma_size_kb:.4f} КБ")
print(f"Дельта-кодирование: размер {delta_size_kb:.4f} КБ")
def bytes_to_bits(byte_data, padding):
    bits = ''.join(f'{byte:08b}' for byte in byte_data)
    return bits[:-padding] if padding > 0 else bits
gamma_bits = bytes_to_bits(gamma_encoded, gamma_padding)
delta_bits = bytes_to_bits(delta_encoded, delta_padding)

print("\nЗакодированные данные в битах:")
print(f"Гамма (первые 100 бит): {gamma_bits[:100]}")
print(f"Дельта (первые 100 бит): {delta_bits[:100]}")

Результаты кодирования:
Гамма-кодирование: размер 9.9365 КБ
Дельта-кодирование: размер 9.2080 КБ

Пример закодированных данных (первые 20 байт):

Закодированные данные в битах:
Гамма (первые 100 бит): 0010111010111111111101010101110101111110101111111001010010100010000000101110001101000100011100001001... (всего 81393 бит)
Дельта (первые 100 бит): 0110111010011111111110100101001110100111111010011111110110101101001000000010101110010010100100000111... (всего 75429 бит)


### Выводы     
Длина битового массива исключений: 1516271 (185 kB)  
Количество исключений (длина массива расстояний): 7401    
Применение методов сжатия к массиву расстояний:   
**Арифметическое кодирование статическое** размер битового закодированного массива = 9.9 kB   
**Арифметическое кодирование динамическое** размер битового закодированного массива = 7.56kB     
**Код Хаффмана** размер сжатых данных = 7.30 kB    
**PPM** размер сжатых данных = 11.0625 kB        
**Гамма-кодирование Элиаса** размер сжатых данных = 9.9365 kB         
**Дельта-кодирование Элиаса** размер сжатых данных = 9.2080 kB      

Таким образом следует применять: динамическое арифметичекое кодирование, так же можно применить кодирование Хаффмана и коды Элиаса (Дельта-кодирование).     

**Передача имен собственных для правил**
Для передачи имен собственных потребуется 150 байт, при этом добавление правила основанного на них сокращает список исключений на 2000 пунктов    


## Задание 2

Для декапитализированного текста построить все контекстные модели 3-го порядка (для каждого контекста собрать всю информацию, как в лекции 7). Посчитать необходимый объём памяти для хранения всех моделей в несжатом виде. Выбрать и обосновать способ кодирования для передачи модели декодеру,указать длину получившегося кода и дать ссылку на использовавшиеся скрипты.

Для каждой подстроки s длины ⩽ k, встретившейся в тексте до текущей
позиции, создадим элемент данных Cs , называемый контекстной моделью и
состоящий из
* строки s
* счетчика c вхождений s в текст (счетчик контекста)
* числа m символов, встречавшихся в тексте с левым контекстом s
* списка символов Ls = [a1, . . . , am], встречавшихся в тексте с левым контекстом s
* счетчика вхождений каждого символа ai в текст с контекстом s (счетчик символа)

Декапитализированный текст

In [ ]:
with open('rousseau-confessions-lowercase.txt', 'r', encoding='utf-8') as file:
    lower_text = file.read()

Контекстные модели $C_s$

Функция для получения контекстных моделей конкретного порядка (3), на ее основе в дальнейшем построим дерево

In [ ]:
def order_context_models(text):
    if not text or len(text) < 4: #  минимальная длина текста для контекста 3
        return {}
    k = 3  # порядок контекста
    raw_models = collections.defaultdict(lambda: collections.defaultdict(int))
    # контекст: text[i:i+k], следующий символ: text[i+k]
    for i in range(len(text) - k):
        context = text[i : i+k]
        char_after = text[i+k]
        raw_models[context][char_after] += 1
    final_models = {}
    for context_str, following_chars_counts in raw_models.items():
        symbol_counts_dict = dict(following_chars_counts)
        context_occurrences_c = sum(symbol_counts_dict.values())
        distinct_symbols_list_Ls = list(symbol_counts_dict.keys())
        distinct_symbols_count_m = len(distinct_symbols_list_Ls)
        # модель C_s
        final_models[context_str] = {
            'context_string': context_str, # 3-символьный контекст s
            'context_occurrences': context_occurrences_c, # количество появлений s
            'distinct_symbols_count': distinct_symbols_count_m, # колво уникальных символов m_s следующих за s
            'distinct_symbols_list': sorted(distinct_symbols_list_Ls), # L_s уникальных символов, следующих за s
            'symbol_counts': symbol_counts_dict # счетчики для каждого символа из L_s следующего за s
        }
    return final_models

Проверка на примере из лекции

In [ ]:
lower_text_content = ""
k_order = 3 # Порядок контекста
lower_text_content = "accaccggacca" # декапитализированный
models = order_context_models(lower_text_content)
if not models:
    print("")
else:
    print(f"{len(models)} уникальных контекстов {k_order}-го порядка\n")
    for i, (context, model_data) in enumerate(models.items()):
        if i < 10:
            print(f"Контекст '{context}':")
            print(f"  Число вхождений контекста: {model_data['context_occurrences']}")
            print(f"  Различных символов после: {model_data['distinct_symbols_count']}")
            print(f"  Список символов: {model_data['distinct_symbols_list']}")
            print(f"  Счетчики символов: {model_data['symbol_counts']}")
        else:
            break


7 уникальных контекстов 3-го порядка

Контекст 'acc':
  Число вхождений контекста: 3
  Различных символов после: 2
  Список символов: ['a', 'g']
  Счетчики символов: {'a': 2, 'g': 1}
Контекст 'cca':
  Число вхождений контекста: 1
  Различных символов после: 1
  Список символов: ['c']
  Счетчики символов: {'c': 1}
Контекст 'cac':
  Число вхождений контекста: 1
  Различных символов после: 1
  Список символов: ['c']
  Счетчики символов: {'c': 1}
Контекст 'ccg':
  Число вхождений контекста: 1
  Различных символов после: 1
  Список символов: ['g']
  Счетчики символов: {'g': 1}
Контекст 'cgg':
  Число вхождений контекста: 1
  Различных символов после: 1
  Список символов: ['a']
  Счетчики символов: {'a': 1}
Контекст 'gga':
  Число вхождений контекста: 1
  Различных символов после: 1
  Список символов: ['c']
  Счетчики символов: {'c': 1}
Контекст 'gac':
  Число вхождений контекста: 1
  Различных символов после: 1
  Список символов: ['c']
  Счетчики символов: {'c': 1}


Теперь построим дерево для простоты с использованием defaultdict

In [ ]:
def full_context_tree(text, k=3):
    from collections import defaultdict
    context_tree = defaultdict(lambda: defaultdict(int))
    n = len(text)
    for i in range(n):
        for l in range(k+1):  # 0 ≤ длина контекста ≤ k
            if i - l < 0 or i >= n:
                continue
            context = text[i - l:i]
            next_char = text[i]
            context_tree[context][next_char] += 1

    return {
        final_models: {
            'context_string': final_models, # 3-символьный контекст s
            'context_occurrences': sum(chars.values()), # количество появлений s
            'distinct_symbols_count': len(chars), # колво уникальных символов m_s следующих за s
            'distinct_symbols_list': sorted(chars), # L_s уникальных символов, следующих за s
            'symbol_counts': dict(chars)  # счетчики для каждого символа из L_s следующего за s
        }
        for final_models, chars in context_tree.items()
    }


Добавим визуализацию    
Для быстрой установки graphviz:       
Download Graphviz from http://www.graphviz.org/download/    
Add below to PATH environment variable (mention the installed graphviz version):    
C:\Program Files (x86)\Graphviz2.38\bin    
C:\Program Files (x86)\Graphviz2.38\bin\dot.exe    
Close any opened Juypter notebook and the command prompt   
Restart Jupyter / cmd prompt and tes    

In [ ]:
from graphviz import Digraph
def visualize_context_tree(models):
    dot = Digraph(comment='Context Tree')
    for context in models:
        dot.node(context, label=f"{context}\n{models[context]['symbol_counts']}")
        if context:
            parent = context[1:]  # родительский контекст — суффикс
            if parent in models:
                dot.edge(parent, context)
    return dot

Добавим также текстовый вывод как было сделано выше

In [ ]:
def print_model_summary(models, max_print=10):
    print(f"{len(models)} контекстов (всех порядков от 0 до 3)\n")
    for i, (context, data) in enumerate(models.items()):
        if i < max_print:
            print(f"Контекст '{context or 'λ'}':")
            print(f"  Число вхождений контекста: {data['context_occurrences']}")
            print(f"  Различных символов после:: {data['distinct_symbols_count']}")
            print(f"  Список символов: {data['distinct_symbols_list']}")
            print(f"  Счетчики символов: {data['symbol_counts']}")
        else:
            break
    if len(models) > max_print:
        print(f"... и еще {len(models) - max_print} моделей\n")


Простая проверка

In [ ]:
text = "accaccggacca"
models = full_context_tree(text, k=3)
print_model_summary(models)
dot = visualize_context_tree(models)
dot.render('context_tree', format='png', cleanup=True)
dot.view()


17 контекстов (всех порядков от 0 до 3)

Контекст 'λ':
  Число вхождений контекста: 12
  Различных символов после:: 3
  Список символов: ['a', 'c', 'g']
  Счетчики символов: {'a': 4, 'c': 6, 'g': 2}
Контекст 'a':
  Число вхождений контекста: 3
  Различных символов после:: 1
  Список символов: ['c']
  Счетчики символов: {'c': 3}
Контекст 'c':
  Число вхождений контекста: 6
  Различных символов после:: 3
  Список символов: ['a', 'c', 'g']
  Счетчики символов: {'c': 3, 'a': 2, 'g': 1}
Контекст 'ac':
  Число вхождений контекста: 3
  Различных символов после:: 1
  Список символов: ['c']
  Счетчики символов: {'c': 3}
Контекст 'cc':
  Число вхождений контекста: 3
  Различных символов после:: 2
  Список символов: ['a', 'g']
  Счетчики символов: {'a': 2, 'g': 1}
Контекст 'acc':
  Число вхождений контекста: 3
  Различных символов после:: 2
  Список символов: ['a', 'g']
  Счетчики символов: {'a': 2, 'g': 1}
Контекст 'ca':
  Число вхождений контекста: 1
  Различных символов после:: 1
  Список симв

'context_tree.pdf'

![image.png](attachment:image.png)

Подсчет объема памяти

Для каждого контекста третьего порядка,
сам контекст s (3 байта)
число ns символов, встречавшихся в контексте
сами встречавшиеся символы (по байту на символ)
частоты встречаемости символов в контексте (ns чисел)

Контекст s - len(s) байт   
Число символов ns - 1 байт   
Список символов - ns байт    
Счетчики символов/список частот - ns * 4 байт



In [ ]:
def model_memory(models):
    total_bytes = 0
    for context, data in models.items():
        context_len = len(context)  # длина контекста  len(s)
        n_s = data['distinct_symbols_count'] # Список символов - ns байт
        context_size = context_len + 1 + 4 * n_s + n_s # общая формула
        total_bytes += context_size
    return total_bytes

Простая проверка

In [ ]:
mem_bytes = model_memory(models)
print(f"Оценка объема памяти: {mem_bytes} байт")

Оценка объема памяти: 173 байт


Сделаем все тоже самое для декапитализированного текста

In [ ]:
with open("rousseau-confessions-lowercase.txt", encoding='utf-8') as f:
    text = f.read()
models = full_context_tree(text, k=3)
print_model_summary(models)
dot = visualize_context_tree(models)
dot.render('context_tree', format='png', cleanup=True)
dot.view()

8141 контекстов (всех порядков от 0 до 3)

Контекст 'λ':
  Число вхождений контекста: 1516271
  Различных символов после:: 51
  Список символов: [' ', '!', '"', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
  Счетчики символов: {'1': 90, '7': 73, '8': 14, '2': 38, ' ': 271827, 't': 112390, 'h': 71591, 'e': 157421, 'c': 33278, 'o': 88806, 'n': 83488, 'f': 29960, 's': 72191, 'i': 90475, 'j': 1552, 'a': 92728, '-': 744, 'q': 1636, 'u': 32622, 'r': 69341, 'b': 16957, 'y': 23751, 'l': 40991, 'd': 52553, 'w': 26481, '.': 8684, 'g': 21149, 'm': 41014, 'k': 5450, '[': 13, ']': 13, 'v': 14255, 'p': 22009, ',': 23710, ';': 3069, 'x': 2432, '"': 613, ':': 701, '!': 299, 'z': 353, "'": 611, '(': 177, ')': 178, '?': 259, '*': 176, '9': 9, '5': 26, '4': 16, '3': 19, '6': 25, '0': 13}
Контекст '1':
  Число вхо

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.0133299 to fit


'context_tree.pdf'

Так как дерево очень большое, прикреплять его здесь не имеет смысла

Посчитаем объем памяти

In [ ]:
mem_bytes = model_memory(models)
print(f"Оценка объема памяти: {mem_bytes} байт")

Оценка объема памяти: 224461 байт


Выполним сжатие модели

Учитывая наличие закономерностей наиболее хорошиее результаты будут давать методы использующие локальные закономерности в символах как PPM

Только для начала нужно сериализовать дерево

In [ ]:
def serialize_models_to_bytes(models):
    json_str = json.dumps(models, ensure_ascii=False, sort_keys=True)
    return json_str.encode('utf-8')  # сериализация в байты


In [ ]:
def deserialize_models_from_bytes(byte_data):
    json_str = byte_data.decode('utf-8')
    return json.loads(json_str)

Проверка

In [ ]:
original = models
serialized = serialize_models_to_bytes(original)
compressed = pyppmd.compress(serialized)
decompressed = pyppmd.decompress(compressed)
restored = deserialize_models_from_bytes(decompressed)

assert original == restored, "дерево не совпадает с оригиналом"
print("сериализация, сжатие, восстановление прошли успешно")
print(f"Размер сериализованного дереваа: {len(serialized)} байт, размер сжатого дерева PPM: {len(compressed)} байт")


сериализация, сжатие, восстановление прошли успешно
Размер сериализованного дереваа: 1632137 байт, размер сжатого дерева PPM: 126731 байт


In [ ]:
len(compressed)/len(serialized)

0.0776472808348809

С помощью PPM получили что сжатая модель составляет примерно 7% от исходной